In [2]:
import numpy as np
import pandas as pd
from collections import deque
import random
import time

In [4]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
STOCK_TO_PREDICT = 'sh500_day'
EPOCHS = 50
BATCH_SIZE = 64
NAME = f'{SEQ_LEN} - SEQ - {FUTURE_PERIOD_PREDICT} - PRED{int(time.time())}'
print(NAME)

60 - SEQ - 3 - PRED1610805754


In [5]:
def signal(current, future):
    if future/current - 1 >= 0.01:
        return True
    else:
        return False

In [8]:
main_df = pd.DataFrame()
stocks = ['000001.SS', '399001.SZ', 'nasdaq']


In [14]:
for stock in stocks:
    print('name:', stock)
    df = pd.read_csv(f'{stock}.csv', names=['Date',
                                              'Open', 'High', 'Low',
                                              'Close', 'Adj Close',
                                              'Volume'])
    print('Lines:', df.shape[0])
    df.rename(columns = {'Close': f'{stock}_close', 'Volume': f'{stock}_volume'}, inplace=True)
    df.set_index('Date', inplace=True)
    df = df[[f'{stock}_close', f'{stock}_volume']]

    if main_df.empty == True:
        main_df = df
    else:
        main_df = main_df.merge(df, left_index=True, right_index=True, how='outer')
main_df.dropna(inplace=True)


name: 000001.SS
Lines: 5213
name: 399001.SZ
Lines: 5213
name: nasdaq
Lines: 5284
